In [ ]:
# Lab: Data Merging and Joining with Pandas (3 Hours)

## Objective:
# - Learn how to merge, join, and concatenate datasets using pandas.
# - Understand different types of joins (inner, left, right, outer).
# - Handle key conflicts, missing values, and track merge origins.
# - Apply merging concepts to a real-world case study.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import warnings
#warnings.filterwarnings("ignore")



In [ ]:
#load data
#Load two dataset student-mat and student-por as math_df and port_df
math_df = pd.read_csv("/content/student-mat.csv")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 14: invalid continuation byte

In [ ]:
# the the head

#display(port_df.head())

**Is head in the right format. If not why not?**

In [ ]:




#load data in correct format


math_df = pd.read_csv("student-mat.csv", delimiter=";") # Specify the delimiter
port_df = pd.read_csv("student-por.csv", delimiter=";") # Specify the delimiter
math_df.head()
#display(port_df.head())


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 14: invalid continuation byte

In [ ]:
display (math_df.head())
display(port_df.head())

# there is a difference between math_df.head() and display(math_df.head()), especially in Jupyter notebooks or environments that support rich output rendering like Google Colab or JupyterLab.

#1. math_df.head() This returns the first 5 rows of the DataFrame.

# In Jupyter notebooks, when it's the last line in a cell, it will be rendered using rich HTML formatting.

# If it's not the last line in a cell, the output might not be shown unless explicitly printed or displayed.

# 2. display(math_df.head())
# The display() function (from IPython) explicitly tells the notebook to render the DataFrame, using rich formatting regardless of where it appears in the cell.

# This is useful when you want to show multiple outputs in the same cell or control exactly when something is displayed.

In [ ]:



# Display the column names of math_df
print("Columns in math_df:")
print(math_df.columns.tolist())

# Display the column names of port_df
print("\nColumns in port_df:")
print(port_df.columns.tolist())

#The .tolist() method in Python (used with NumPy arrays or Pandas Series/DataFrames) converts the data structure into a regular Python list.


# Actity 1 : Merge

In [ ]:

# Task 1: Merge on common columns (inner join)

merged_df = pd.merge(math_df, port_df, on=["school", "sex", "age"], how="inner", indicator=True)
print(merged_df.shape)
print(merged_df['_merge'].value_counts())

#Merges math_df and port_df using an inner join.

#The merge is based on matching values in the columns: "school", "sex", and "age".

#how="inner" means only rows with matching values in both dataframes will be included in the result.

#indicator=True adds a special column _merge to show the origin of each row:

#'left_only' → only in math_df

#'right_only' → only in port_df

#'both' → matched in both


In [ ]:
print(merged_df.shape)

In [ ]:
display(merged_df.head())


In [ ]:



#1. **`inner join` on `school`, `sex`, and `age`:**  An inner join only includes
#rows where the values of *all* specified join keys (`school`, `sex`, and `age`)
# match in *both* dataframes.  If a student exists in `math_df` but has a different
# value for any of these keys in `port_df`, or vice-versa, that student's record will be excluded from the merged result.

#2. **Column name conflicts:**  Because several columns share names between the two dataframes
# (e.g., `absences`, `G1`, `G2`, `G3`), Pandas appends `_x` to columns from the left dataframe (`math_df`) and `_y` to columns from the right dataframe (`port_df`) when merging to create unique column names.  This is what you are seeing with `absences_y` and `absences_x`. This means `absences_y` refers to the `absences` value from `port_df`.


#3. **First five rows may not match:**  While the first few rows of `math_df` and
#`port_df` might seem to have similar values for `school`, `sex`, and `age`, the
#crucial point is that an inner join requires *exact* matches across *all three* columns. A slight difference in one of those columns (even in a row not shown by `.head()`) for a given student in either dataframe, will prevent that student's data from being included in the merged output. The merged DataFrame's first five rows are the first five rows where the merge keys fully match between both dataframes.


#**In summary:** The `merged_df` displays different values for `absences`
 #(and other duplicated columns) because it's showing the value of `absences`
 #from `port_df` (`absences_y`). The first five rows of the merged dataset do
 # not correspond directly to the first five rows of the original dataframes because of the `inner join` logic which filters out rows that don't have exact matches on all the specified merge columns.  The resulting `merged_df` only shows the rows that have a perfect match for `school`, `sex`, and `age` in both `math_df` and `port_df`.


In [ ]:
# Task 2: Merge with different column names
port_df_renamed = port_df.rename(columns={"age": "student_age"})
merged_custom = pd.merge(math_df, port_df_renamed, left_on="age", right_on="student_age", how="inner")
print(merged_custom.head())

#Merges math_df and the renamed port_df_renamed.

#left_on="age": Use the age column from math_df.

#right_on="student_age": Use the student_age column from port_df_renamed.

#how="inner": Only rows where both age and student_age match will be included.
#Sometimes, datasets use different naming conventions even for the same data.
#This technique lets you merge them by explicitly matching the relevant columns.
#examples "age": "student_age",     "school": "school_name",     "sex": "gender"

In [ ]:
# show column names of merged_custom.df

print(merged_custom.columns.tolist())


In [ ]:

# Task 3: Observe effect of duplicate keys
math_dup = math_df.iloc[:10].copy()
math_dup = pd.concat([math_dup, math_dup], ignore_index=True)
merged_dup = pd.merge(math_dup, port_df, on=["school", "sex", "age"], how="inner")
print(merged_dup.shape)  # many-to-many merge

In [ ]:
# prompt: briefly explain what does this command do "# Task 3: Observe effect of duplicate keys
# math_dup = math_df.iloc[:10].copy()
# math_dup = pd.concat([math_dup, math_dup], ignore_index=True)
# merged_dup = pd.merge(math_dup, port_df, on=["school", "sex", "age"], how="inner")
# print(merged_dup.shape)  # many-to-many merge"

# This code demonstrates the effect of duplicate keys on a many-to-many merge operation using Pandas.

#1. **Duplicate Creation:** It takes the first 10 rows of the `math_df` DataFrame and creates a copy called `math_dup`. Then, it concatenates `math_dup` with itself, effectively doubling the rows and creating duplicate entries.  The `ignore_index=True` argument resets the index of the concatenated DataFrame.

#2. **Many-to-Many Merge:** It then performs an inner merge between the modified `math_dup` (which now has duplicate rows) and the `port_df` DataFrame based on the columns "school", "sex", and "age". Because `math_dup` has duplicates and `port_df` may also have matching rows on the join columns, this creates a many-to-many merge scenario.  For each duplicate row in `math_dup` that finds a match in `port_df`, a new row will be created in the `merged_dup` DataFrame.

#3. **Shape Observation:**  Finally, it prints the shape (number of rows and columns) of the resulting `merged_dup` DataFrame.  The shape will be larger than the shape of `port_df` because of the duplication in `math_dup`. The number of rows in the `merged_dup` DataFrame would be equal to the number of times each duplicated row in `math_dup` is matched in `port_df`.

#In essence, this section illustrates how duplicate keys in a merge operation
#can lead to an increase in the number of rows in the resulting merged DataFrame, reflecting a many-to-many relationship between the dataframes being joined.


In [ ]:
# ---------------------------
# Lab Activity 2: Join Types
# ---------------------------

# Create sample HR datasets
#https://github.com/pouyasattari/HR-Dataset-Analysis/blob/main/HRDataset_v14.csv
hr_df = pd.read_csv("/content/HRDataset_v14.csv")

In [ ]:
departments = hr_df[['Department', 'Employee_Name']].drop_duplicates()
training_scores = pd.DataFrame({
    'Employee_Name': ['Douglas', 'Katelyn', 'John', 'Zoe'],
    'Training_Score': [88, 92, 75, 69]
})
#hr_df[['Department', 'Employee_Name']]
# Selects just the Department and Employee_Name columns from the HR dataset.

#.drop_duplicates()
# Removes duplicate rows where the same employee appears more than once in
#the same department.

#Stores result in departments
# Now you have a simplified DataFrame showing unique employee-department
#pairs.

#--------------
#training_scores = pd.DataFrame({    'Employee_Name': ['Douglas', 'Katelyn', 'John', 'Zoe'],'Training_Score': [88, 92, 75, 69]})
#This creates a new DataFrame manually containing training scores for a few employees.

#You're preparing two datasets that can be merged later.

#One shows which department each employee belongs to.

#The other shows each employee's training score.


In [ ]:
hr_df.head()

In [ ]:
# Show top 5 rows of the departments


In [ ]:
#Show head of the training scores

In [ ]:
# Task 1: Inner Join
inner_joined = pd.merge(hr_df, training_scores, on="Employee_Name", how="inner")

#This command performs an inner join between hr_df and training_scores on the Employee_Name column, keeping only the rows where the employee exists in both DataFrames.

In [ ]:

# Task 2: Left Join with NaN handling
left_joined = pd.merge(hr_df, training_scores, on="Employee_Name", how="left")
left_joined['Training_Score'] = left_joined['Training_Score'].fillna(0)

#Perform a left join between hr_df and training_scores on Employee_Name,
#keeping all records from hr_df and adding Training_Score where available.

#Replace missing (NaN) training scores with 0 to handle unmatched employees
#who didn’t have a training score.


In [ ]:
left_joined.head()

In [ ]:

# Task 3: Outer Join
outer_joined = pd.merge(hr_df, training_scores, on="Employee_Name", how="outer", indicator=True)
print(outer_joined['_merge'].value_counts())

#Perform an outer join between hr_df and training_scores on Employee_Name,
#keeping all records from both DataFrames whether they match or not.
#Count and print how many rows came from both DataFrames (both),
#only from hr_df (left_only), or only from training_scores (right_only) using the _merge indicator column.

In [ ]:

# ---------------------------
# Lab Activity 3: Case Study
# ---------------------------

# Case: Unify HR demographics, departments, and training into a clean profile
employee_df = hr_df[['Employee_Name', 'Department', 'Position', 'Sex', 'MaritalDesc']]
final_profile = pd.merge(employee_df, training_scores, on="Employee_Name", how="left")

In [ ]:





# Clean up missing scores
final_profile['Training_Score'] = final_profile['Training_Score'].fillna("Not Participated")



In [ ]:
# Show result
print(final_profile.head())


In [ ]:
# Practice Activities

# Activity 1: Exploring Different Merge Types

# Objective:  Practice merging two datasets using various join types (inner, left, right, outer) and observe the differences in the resulting dataframes.

# Instructions:
# 1. Load the "student-mat.csv" and "student-por.csv" datasets into pandas DataFrames (remember to specify the delimiter ';').
# 2. Merge the two dataframes using each of the four merge types (inner, left, right, outer) on the common columns "school", "sex", and "age".
# 3. For each merged dataframe, observe:
#    - The number of rows and columns.
#    - The content of the first few rows.
#    - The unique values and counts in the '_merge' indicator column (if used).  How do these counts explain the rows in the merged data?
# 4. Explain the differences between the merge types based on your observations.
# 5. Answer the following questions:
#     - Which merge type is most suitable for this scenario where you're combining academic data from two sources?
#     - Which merge type preserves information from only one dataset, and which one from both?  How is "information" preserved in a merge?  Is it a row or a column or something else?
#     - How does using `indicator=True` help you understand the merge results?
# 6. Submit your code and analysis (explanations) in a Jupyter Notebook.

# Activity 2: Handling Missing Values and Duplicate Keys

# Objective:  Practice merging datasets with missing values and duplicate keys and apply appropriate strategies for handling these situations.

# Instructions:
# 1. Use the provided HR datasets (`hr_df`, `departments`, and `training_scores`) or create similar datasets with some missing values in the `Training_Score` column and duplicate employee names.
# 2. Perform a left join between `hr_df` and `training_scores` on "Employee_Name" with missing values filled (NaNs). Experiment with different fill methods.
# 3. Create duplicate rows within one of the dataframes that are merged. Perform the same left join. What changes in your outcome? How many rows are in each merge outcome?
# 4. Perform an inner join between the same two dataframes. Note how many rows are returned. Compare the output of the inner join to the output of the left join with missing values filled. Why might the inner join output a different number of rows?
# 5. Explore the effect of using different join keys.
# 6.  What are the trade-offs between using an inner join versus a left/right join?
# 7. Submit your code and observations/explanations in a Jupyter Notebook.
